In [2]:
import numpy as np
import sympy as sp
from scipy.stats import unitary_group

In [3]:
i = sp.I

In [4]:
def generateComplexVector(N):
    real_parts = np.random.randint(-10, 10, N)
    imaginary_parts = np.random.randint(-10, 10, N)
    complex_vector = real_parts + i * imaginary_parts
    return complex_vector

def generateComplexMatrix(N):
    return sp.Matrix([generateComplexVector(N) for _ in range(N)])

def complexProjection(u,v):
    return ((u.H.dot(v))/(u.H.dot(u)) * u).expand()

def GramSchmidt(M):
    matrix = M
    for j in range(1, M.shape[0]):
        col = matrix.col(j)
        for k in range(0, j):
            col -= complexProjection(matrix.col(k),col)
        matrix[j] = col.normalized()
    matrix[0] = matrix.col(0).normalized()
    return matrix.expand()

In [5]:
def generateQuantumDynamics(N):
    return GramSchmidt(generateComplexMatrix(N))

In [6]:
def checkIfQuantumDynamics(M):
    return (M@M.H).expand().equals(sp.Matrix(np.eye(M.shape[0]), dtype=np.int8))

In [7]:
M = generateQuantumDynamics(3)
display(M, (M@M.H).expand(), checkIfQuantumDynamics(M))

Matrix([
[2*sqrt(93)/93 - 2*sqrt(93)*I/93, -308*sqrt(18786)/46965 - 67*sqrt(18786)*I/31310,   -9*sqrt(17210)/43025 - 91*sqrt(17210)*I/86050],
[                  2*sqrt(93)/31,   -3*sqrt(18786)/15655 + 41*sqrt(18786)*I/31310, 136*sqrt(17210)/43025 + 419*sqrt(17210)*I/86050],
[                7*sqrt(93)*I/93,  -64*sqrt(18786)/46965 + 67*sqrt(18786)*I/46965, 164*sqrt(17210)/43025 - 127*sqrt(17210)*I/43025]])

Matrix([
[1, 0, 0],
[0, 1, 0],
[0, 0, 1]])

True

In [8]:
def calcState(matrix, start, k):
    state = start
    for _ in range(abs(k)):
        state = matrix @ state if k > 0 else matrix.T @ state
    return state

In [9]:
def calculateTransitionProbability(M, i, j, T):
    iDirac = np.zeros((M.shape[0], 1))
    iDirac[i] = 1
    state = calcState(M, iDirac, T)
    state = state.normalized()
    return sp.Abs(state[j])**2

In [10]:
M = sp.Matrix([[1, 1], [1, -1]])*1/sp.sqrt(2)
calculateTransitionProbability(M, 0, 1, -3)

0.500000000000000

In [11]:
M = generateQuantumDynamics(4)
display(M)

Matrix([
[-5*sqrt(278)/139 - 3*sqrt(278)*I/278,   1313*sqrt(21556954)/21556954 - 778*sqrt(21556954)*I/10778477, -689709*sqrt(307900455358)/615800910716 + 159863*sqrt(307900455358)*I/615800910716,  -2338649*sqrt(44804011093781)/224020055468905 - 4099239*sqrt(44804011093781)*I/448040110937810],
[  -3*sqrt(278)/139 - sqrt(278)*I/278, -2625*sqrt(21556954)/21556954 + 1327*sqrt(21556954)*I/10778477,   -175223*sqrt(307900455358)/615800910716 - 5899*sqrt(307900455358)*I/615800910716,  -188493*sqrt(44804011093781)/224020055468905 - 29597473*sqrt(44804011093781)*I/448040110937810],
[-5*sqrt(278)/278 - 7*sqrt(278)*I/278,  -275*sqrt(21556954)/21556954 + 1419*sqrt(21556954)*I/21556954,  231549*sqrt(307900455358)/615800910716 + 346495*sqrt(307900455358)*I/615800910716,  8768767*sqrt(44804011093781)/448040110937810 + 23156353*sqrt(44804011093781)*I/224020055468905],
[ 3*sqrt(278)/278 + 7*sqrt(278)*I/278,   -771*sqrt(21556954)/21556954 + 891*sqrt(21556954)*I/21556954, -674295*sqrt(307900455358)/615800

In [12]:
d0 = calculateTransitionProbability(M, 0, 0, -1).simplify().evalf(6)
d1 = calculateTransitionProbability(M, 0, 1, -1).simplify().evalf(6)
d2 = calculateTransitionProbability(M, 0, 2, -1).simplify().evalf(6)
d3 = calculateTransitionProbability(M, 0, 3, -1).simplify().evalf(6)
display(d0,d1,d2,d3, sum([d0, d1, d2, d3]))

0.392086

0.192286

0.406994

0.00863335

1.00000

In [20]:
Hm = sp.Matrix([
    [1, 1],
    [1, -1]
])/sp.sqrt(2)
zero = sp.Matrix([1, 0])

state = calcState(Hm, zero, 3)
backInTime = calcState(Hm, state, -3)
display(Hm, zero, state, backInTime)

Matrix([
[sqrt(2)/2,  sqrt(2)/2],
[sqrt(2)/2, -sqrt(2)/2]])

Matrix([
[1],
[0]])

Matrix([
[sqrt(2)/2],
[sqrt(2)/2]])

Matrix([
[1],
[0]])